In [ ]:
import os
import sys
import gzip
import shutil
import pandas as pd
from tqdm import tqdm

if os.getcwd().replace("algorithm_development", "utils") not in sys.path:
    sys.path.append(os.getcwd().replace("algorithm_development", "utils"))

import parse_fit
pd.options.display.max_columns = 100

%load_ext autoreload
%autoreload 2

In [ ]:
#######################
### UNZIP ALL FILES ###
#######################
strava_export_path = "\\".join(os.getcwd().split("\\")[:-1]) + "\\export_105647830"

for filename in os.listdir(strava_export_path + "\\activities"):
    if filename.endswith('.gz') and filename.replace(".gz", "") not in os.listdir(strava_export_path + "\\activities"): 
        print(filename)
        with gzip.open("\\".join((strava_export_path + "\\activities", filename)), 'rb') as f_in:
            with open("\\".join((strava_export_path + "\\activities", filename))[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [ ]:
activities_df = pd.read_csv(f"{strava_export_path}\\activities.csv")
activities_df["Filename"] = activities_df["Filename"].apply(lambda v: v.split("/")[1].replace(".gz", "")
                                                            if isinstance(v, str) else v)


list_of_fit_files = ["\\".join((strava_export_path + "\\activities", v))
                     for v in os.listdir(strava_export_path + "\\activities")
                     if ".gpx" not in v and ".gz" not in v]
list_of_gpx_files = ["\\".join((strava_export_path + "\\activities", v))
                     for v in os.listdir(strava_export_path + "\\activities")
                     if ".gpx" in v]

In [ ]:
##########################################
### GET TO DATAFRAME PER EACH FIT FILE ###
##########################################
list_of_dfs_fit_points = []
list_of_dfs_fit_laps = []
for file in tqdm(list_of_fit_files):
    try:
        d_ = parse_fit.get_dataframes(file)
        list_of_dfs_fit_laps.append(d_[0])
        list_of_dfs_fit_points.append(d_[1])
    except:
        print(file)

fit_file_laps = pd.concat(list_of_dfs_fit_laps, ignore_index = True)
fit_file_points = pd.concat(list_of_dfs_fit_points, ignore_index = True)

fit_file_points = fit_file_points.merge(activities_df, left_on = "FileName", right_on = "Filename", how = "left")
fit_file_points = fit_file_points.drop("compressed_speed_distance", axis = 1)

fit_file_laps = fit_file_laps.merge(activities_df, left_on = "FileName", right_on = "Filename", how = "left")

# SAVE FILES TO AVOID RECOMPUTING EACH TIME
fit_file_laps.to_parquet("{}\\fit_file_laps.parquet".format("\\".join(strava_export_path.split("\\")[:-1]) + "\\data\\"))
fit_file_points.to_parquet("{}\\fit_file_points.parquet".format("\\".join(strava_export_path.split("\\")[:-1]) + "\\data\\"))

---
---
---